# Downloads and necessary libraries

In [51]:
import numpy as np

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [6]:
!pip install selenium undetected-chromedriver

# Importing the merged dataset

In [52]:
df = pd.read_csv('data/final_dataset/df_final.csv', low_memory=False)

In [53]:
df.shape

(30161, 42)

In [54]:
df.head()

,ATP,Location,Tournament,Date,Series,Court,Surface,Round,Best of,Winner,Loser,WRank,LRank,WPts,LPts,W1,L1,W2,L2,W3,L3,W4,L4,W5,L5,Wsets,Lsets,Comment,B365W,B365L,EXW,EXL,LBW,LBL,PSW,PSL,SJW,SJL,MaxW,MaxL,AvgW,AvgL
0,1,Brisbane,Brisbane International,2012-12-31,ATP250,Outdoor,Hard,1st Round,3.0,Mayer F.,Giraldo S.,28.0,57.0,1215.0,778.0,6.0,4.0,6.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,Completed,1.36,3.00,1.45,2.65,1.44,2.62,1.47,2.85,1.44,2.63,1.47,3.20,1.42,2.78
1,3,Doha,Qatar Exxon Mobil Open,2012-12-31,ATP250,Outdoor,Hard,1st Round,3.0,Davydenko N.,Hanescu V.,44.0,62.0,885.0,753.0,6.0,2.0,6.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,Completed,1.33,3.25,1.35,3.00,1.36,3.00,1.29,3.92,1.33,3.00,1.38,3.92,1.33,3.21
2,3,Doha,Qatar Exxon Mobil Open,2012-12-31,ATP250,Outdoor,Hard,1st Round,3.0,Zemlja G.,Fognini F.,55.0,45.0,782.0,880.0,6.0,2.0,3.0,6.0,6.0,4.0,NaN,NaN,NaN,NaN,2.0,1.0,Completed,1.57,2.25,1.65,2.15,1.67,2.10,1.73,2.22,1.67,2.10,1.73,2.40,1.63,2.21
3,3,Doha,Qatar Exxon Mobil Open,2012-12-31,ATP250,Outdoor,Hard,1st Round,3.0,Troicki V.,Phau B.,38.0,75.0,1055.0,668.0,6.0,3.0,3.0,6.0,6.0,4.0,NaN,NaN,NaN,NaN,2.0,1.0,Completed,1.36,3.00,1.35,3.00,1.33,3.25,1.38,3.26,1.44,2.63,1.48,3.95,1.40,2.87
4,3,Doha,Qatar Exxon Mobil Open,2012-12-31,ATP250,Outdoor,Hard,1st Round,3.0,Kohlschreiber P.,Dodig I.,20.0,72.0,1770.0,695.0,7.0,6.0,6.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,Completed,1.44,2.62,1.5,2.50,1.50,2.50,1.54,2.63,1.50,2.50,1.57,3.00,1.50,2.52


In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30161 entries, 0 to 30160
Data columns (total 42 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ATP         30161 non-null  int64  
 1   Location    30161 non-null  object 
 2   Tournament  30161 non-null  object 
 3   Date        30161 non-null  object 
 4   Series      30161 non-null  object 
 5   Court       30161 non-null  object 
 6   Surface     30161 non-null  object 
 7   Round       30161 non-null  object 
 8   Best of     30146 non-null  float64
 9   Winner      30161 non-null  object 
 10  Loser       30161 non-null  object 
 11  WRank       30149 non-null  float64
 12  LRank       30098 non-null  float64
 13  WPts        30150 non-null  float64
 14  LPts        30098 non-null  float64
 15  W1          29958 non-null  float64
 16  L1          29961 non-null  float64
 17  W2          29714 non-null  float64
 18  L2          29714 non-null  float64
 19  W3          14507 non-nul

# Preparing the data for modeling

In [56]:
# Column mapping
prefix_map = {
    'WRank': 'rank',
    'LRank': 'rank',
    'WPts': 'pts',
    'LPts': 'pts',
    'W1': 'set1',
    'L1': 'set1',
    'W2': 'set2',
    'L2': 'set2',
    'W3': 'set3',
    'L3': 'set3',
    'W4': 'set4',
    'L4': 'set4',
    'W5': 'set5',
    'L5': 'set5',
    'Wsets': 'sets',
    'Lsets': 'sets',
    'B365W': 'B365',
    'B365L': 'B365',
    'EXW': 'EX',
    'EXL': 'EX',
    'LBW': 'LB',
    'LBL': 'LB',
    'PSW': 'PS',
    'PSL': 'PS',
    'SJW': 'SJ',
    'SJL': 'SJ',
    'MaxW': 'Max',
    'MaxL': 'Max',
    'AvgW': 'Avg',
    'AvgL': 'Avg',
}

# Randomly assign who is player_A and who is player_B
mask = np.random.rand(len(df)) < 0.5
df['player_A'] = np.where(mask, df['Winner'], df['Loser'])
df['player_B'] = np.where(mask, df['Loser'], df['Winner'])
df['target'] = np.where(mask, 1, 0)  # 1 if player_A is winner

# Initialize columns to drop
columns_to_drop = ['Winner', 'Loser']

# Create player_A and player_B feature columns
for col, newcol in prefix_map.items():
    if col.endswith('W'):
        col_w, col_l = col, col[:-1] + 'L'
    elif col.endswith('L'):
        col_l, col_w = col, col[:-1] + 'W'
    else:
        continue

    if col_w in df.columns and col_l in df.columns:
        df[f'P_A_{newcol}'] = np.where(mask, df[col_w], df[col_l])
        df[f'P_B_{newcol}'] = np.where(mask, df[col_l], df[col_w])
        columns_to_drop.extend([col_w, col_l])

# Drop original winner/loser and their stat columns
df = df.drop(columns=columns_to_drop)

# Optional: reorder columns for readability
ordered_cols = ['player_A', 'player_B', 'target'] + [col for col in df.columns if col not in ['player_A', 'player_B', 'target']]
df = df[ordered_cols]

In [57]:
df.head()

,player_A,player_B,target,ATP,Location,Tournament,Date,Series,Court,Surface,Round,Best of,WRank,LRank,WPts,LPts,W1,L1,W2,L2,W3,L3,W4,L4,W5,L5,Wsets,Lsets,Comment,P_A_B365,P_B_B365,P_A_EX,P_B_EX,P_A_LB,P_B_LB,P_A_PS,P_B_PS,P_A_SJ,P_B_SJ,P_A_Max,P_B_Max,P_A_Avg,P_B_Avg
0,Giraldo S.,Mayer F.,0,1,Brisbane,Brisbane International,2012-12-31,ATP250,Outdoor,Hard,1st Round,3.0,28.0,57.0,1215.0,778.0,6.0,4.0,6.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,Completed,3.00,1.36,2.65,1.45,2.62,1.44,2.85,1.47,2.63,1.44,3.20,1.47,2.78,1.42
1,Hanescu V.,Davydenko N.,0,3,Doha,Qatar Exxon Mobil Open,2012-12-31,ATP250,Outdoor,Hard,1st Round,3.0,44.0,62.0,885.0,753.0,6.0,2.0,6.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,Completed,3.25,1.33,3.0,1.35,3.00,1.36,3.92,1.29,3.00,1.33,3.92,1.38,3.21,1.33
2,Zemlja G.,Fognini F.,1,3,Doha,Qatar Exxon Mobil Open,2012-12-31,ATP250,Outdoor,Hard,1st Round,3.0,55.0,45.0,782.0,880.0,6.0,2.0,3.0,6.0,6.0,4.0,NaN,NaN,NaN,NaN,2.0,1.0,Completed,1.57,2.25,1.65,2.15,1.67,2.10,1.73,2.22,1.67,2.10,1.73,2.40,1.63,2.21
3,Troicki V.,Phau B.,1,3,Doha,Qatar Exxon Mobil Open,2012-12-31,ATP250,Outdoor,Hard,1st Round,3.0,38.0,75.0,1055.0,668.0,6.0,3.0,3.0,6.0,6.0,4.0,NaN,NaN,NaN,NaN,2.0,1.0,Completed,1.36,3.00,1.35,3.0,1.33,3.25,1.38,3.26,1.44,2.63,1.48,3.95,1.40,2.87
4,Kohlschreiber P.,Dodig I.,1,3,Doha,Qatar Exxon Mobil Open,2012-12-31,ATP250,Outdoor,Hard,1st Round,3.0,20.0,72.0,1770.0,695.0,7.0,6.0,6.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,Completed,1.44,2.62,1.5,2.5,1.50,2.50,1.54,2.63,1.50,2.50,1.57,3.00,1.50,2.52


# Adding player info

### Fixing the player names (abreviated names -> full names) to scrape player info

In [58]:
import unicodedata
import importlib
import re
import full_names_list

importlib.reload(full_names_list)
from full_names_list import full_names

def normalize(text):
    if not isinstance(text, str):
        return ""
    text = text.strip()
    text = re.sub(r'\.+$', '', text)
    text = unicodedata.normalize('NFD', text)
    text = ''.join(c for c in text if unicodedata.category(c) != 'Mn')
    text = text.lower()
    text = text.replace('-', ' ')
    text = text.replace("'", "")
    text = re.sub(r'\bo\s+connell\b', 'oconnell', text)
    text = re.sub(r'\s+', ' ', text)
    return text

def generate_initials_variants(parts):
    initials = []
    for p in parts:
        p_clean = re.sub(r'[^A-Za-z]', '', p)
        if p_clean:
            initials.append(p_clean[0].upper())
    variants = [
        '.'.join(initials) + '.',       # J.P.
        ''.join(initials) + '.',        # JP.
        '. '.join(initials) + '.',      # J. P.
        initials[0] + '.'               # J.
    ]
    return variants

lookup = {}

for full_name in full_names:
    parts = full_name.split()
    if len(parts) < 2:
        continue

    for i in range(1, len(parts)):
        first_parts = parts[:i]
        last_parts = parts[i:]

        initials_variants = generate_initials_variants(first_parts)

        # Generate for full last name
        for initials in initials_variants:
            key = normalize(f"{' '.join(last_parts)} {initials}")
            lookup[key] = full_name

        # Generate for truncated last name (last word only)
        last_word = last_parts[0]
        for initials in initials_variants:
            key_short = normalize(f"{last_word} {initials}")
            lookup[key_short] = full_name

    # Special case Estrella Burgos
    if normalize(full_name).startswith("victor estrella"):
        lookup[normalize("estrella burgos v.")] = full_name

    # Special case Galan
    if normalize(full_name).startswith("daniel elahi galan riveros"):
        initials_variants = generate_initials_variants(parts[:-2])
        last_word = parts[-2]
        for initials in initials_variants:
            key_galan = normalize(f"{last_word} {initials}")
            lookup[key_galan] = full_name

    # Special case Haider-Maurer
    if normalize(full_name).startswith("mario haider maurer"):
        lookup[normalize("haider maurer a.")] = full_name

# Apply mapping function:
def map_name(name):
    if not isinstance(name, str):
        return None

    name_norm = normalize(name)
    parts = name_norm.split()
    if len(parts) < 2:
        return lookup.get(name_norm)

    last_name = ' '.join(parts[:-1])
    raw_initials = parts[-1].replace('.', '')

    candidates = [
        normalize(f"{last_name} {raw_initials[0]}."),             
        normalize(f"{last_name} {'.'.join(raw_initials)}."),       
        normalize(f"{last_name} {''.join(raw_initials)}."),        
        normalize(f"{last_name} {' '.join([c+'.' for c in raw_initials])}"),
        normalize(name_norm)
    ]

    for cand in candidates:
        if cand in lookup:
            return lookup[cand]
    return None

df['player_A_full'] = df['player_A'].apply(map_name)
df['player_B_full'] = df['player_B'].apply(map_name)

In [59]:
unmatched_A = df[df['player_A_full'].isna()]['player_A'].unique().tolist()
print(", ".join(unmatched_A))

Stebe C-M., Ali Mutawa J.M., Jones G., Authom M., Baker J., Mektic N., Serra F., Naso G., Brugues-Davi A., Marcan D., Mello R., Sweeting R., Velotti A., Garza D., Ramirez C., Reyes-Varela M.A., Veic A., Ouanna J., Balleret B., Granollers G., Lopez M., Desein N., Marti J., Starace P., Nielsen F., Ghedin R., Bopanna R., Bohli S., Knittel B., Reynolds B., Eriksson M., Beck A., Vinciguerra A., Kern R., Korolev E., Guccione C., King K., Fischer M., Setkic A., Peliwo F., Altamirano C., Trongcharoenchaikul W., Langer N., Nedovyesov O., Pospisil J., Zayed M.S., Statham J., Guez D., Delic M., Artunedo Martinavarro A., Kosakowski D., Khaddari H., Machado R., Vaisse M., Cox D., Dustov F., Puetz T., Lindell C., Davydenko P., Marti Y., Struvay E., Cabal J.S., Galovic V., Gao X., Jasika O., Ouyang B., Takeuchi K., Herbert P-H., Bai Y., Wang C., Rosenholm P., Baluda V., Krstin P., Prashanth V., Androic T., Lapentti G., Martin F., Ouahab L., Idmbarek Y., Silva F., Ledovskikh M., Trinker B., Vega Herna

In [60]:
# Get all unmatched 'player_A' and 'player_B' names
unmatched = pd.concat([
    df[df['player_A_full'].isna()]['player_A'],
    df[df['player_B_full'].isna()]['player_B']
])

# Count occurrences and sort descending
name_counts = unmatched.value_counts()

# Convert to list (most frequent first)
ordered_unmatched_list = name_counts.index.tolist()

# Print for copy-paste
for name in ordered_unmatched_list:
    print(name)

Meligeni Alves F.
Delic M.
Riedi L.
Shimabukuro S.
Wong C.
Borg L.
Kwiatkowski T.S.
Landaluce M.
Lindell C.
Beck A.
Klein B.
Tien L.
King K.
Galovic V.
Li Z.
Mektic N.
Lazarov A.
Ouahab L.
Marcora R.
Zhou Y.
Janvier M.
Masur D.
Peliwo F.
Weintraub A.
Reynolds B.
Sweeny D.
Sarkissian A.
Bourgue M.
Heide G.
Ojeda Lara R.
Wu T.L.
Veic A.
Quiroz R.
Hsu Y.
Blancaneaux G.
Gomez L.
Svrcina D.
Jasika O.
Marti Y.
Kumar O.
Petrovic D.
Machado R.
Krstin P.
Statham J.
Whittington A.
Schoolkate T.
Squire H.
Serra F.
Pacheco Mendez R.
Agamenone F.
Topo M.
Starace P.
Hong S.
Sachko V.
Bai Y.
Fearnley J.
Ghem A.
Yevseyev D.
Rocha H.
Nikles J.
Vacherot V.
Ahouda A.
Burruchaga R.
Valkusz M.
Uchida K.
Ward A.
Spizzirri E.
Lapentti G.
Barrientos N.
Droguet T.
Korolev E.
Kolar Z.
Hassan B.
Piros Z.
Lama G.
Catarina L.
Sakharov G.
Rehberg M.
Gakhov I.
Torpegaard M.
Eriksson M.
Kadhe A.
Mccabe J.
Dustov F.
Gray A.
Guez D.
Fischer M.
King E.
Lopez Villasenor G.
Wessels L.
El Amrani R.
Silva F.F.
Lajal M.
Oliv

In [61]:
df.head()

,player_A,player_B,target,ATP,Location,Tournament,Date,Series,Court,Surface,Round,Best of,WRank,LRank,WPts,LPts,W1,L1,W2,L2,W3,L3,W4,L4,W5,L5,Wsets,Lsets,Comment,P_A_B365,P_B_B365,P_A_EX,P_B_EX,P_A_LB,P_B_LB,P_A_PS,P_B_PS,P_A_SJ,P_B_SJ,P_A_Max,P_B_Max,P_A_Avg,P_B_Avg,player_A_full,player_B_full
0,Giraldo S.,Mayer F.,0,1,Brisbane,Brisbane International,2012-12-31,ATP250,Outdoor,Hard,1st Round,3.0,28.0,57.0,1215.0,778.0,6.0,4.0,6.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,Completed,3.00,1.36,2.65,1.45,2.62,1.44,2.85,1.47,2.63,1.44,3.20,1.47,2.78,1.42,Santiago Giraldo,Florian Mayer
1,Hanescu V.,Davydenko N.,0,3,Doha,Qatar Exxon Mobil Open,2012-12-31,ATP250,Outdoor,Hard,1st Round,3.0,44.0,62.0,885.0,753.0,6.0,2.0,6.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,Completed,3.25,1.33,3.0,1.35,3.00,1.36,3.92,1.29,3.00,1.33,3.92,1.38,3.21,1.33,Victor Hănescu,Nikolay Davydenko
2,Zemlja G.,Fognini F.,1,3,Doha,Qatar Exxon Mobil Open,2012-12-31,ATP250,Outdoor,Hard,1st Round,3.0,55.0,45.0,782.0,880.0,6.0,2.0,3.0,6.0,6.0,4.0,NaN,NaN,NaN,NaN,2.0,1.0,Completed,1.57,2.25,1.65,2.15,1.67,2.10,1.73,2.22,1.67,2.10,1.73,2.40,1.63,2.21,Grega Zemlja,Fabio Fognini
3,Troicki V.,Phau B.,1,3,Doha,Qatar Exxon Mobil Open,2012-12-31,ATP250,Outdoor,Hard,1st Round,3.0,38.0,75.0,1055.0,668.0,6.0,3.0,3.0,6.0,6.0,4.0,NaN,NaN,NaN,NaN,2.0,1.0,Completed,1.36,3.00,1.35,3.0,1.33,3.25,1.38,3.26,1.44,2.63,1.48,3.95,1.40,2.87,Viktor Troicki,Bjorn Phau
4,Kohlschreiber P.,Dodig I.,1,3,Doha,Qatar Exxon Mobil Open,2012-12-31,ATP250,Outdoor,Hard,1st Round,3.0,20.0,72.0,1770.0,695.0,7.0,6.0,6.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,Completed,1.44,2.62,1.5,2.5,1.50,2.50,1.54,2.63,1.50,2.50,1.57,3.00,1.50,2.52,Philipp Kohlschreiber,Ivan Dodig


In [ ]:
# # Save the cleaned dataset
# df.to_csv("data/matches_df_final.csv", index=False)

### Scrapping player info

In [ ]:
# pd.concat([df['player_A_full'], df['player_B_full']]).nunique()

475

In [ ]:
# # Get all unmatched 'player_A' and 'player_B' names
# unmatched = pd.concat([
#     df[df['player_A_full'].isna() == False]['player_A_full'],
#     df[df['player_B_full'].isna() == False]['player_B_full']
# ])

# # Count occurrences and sort descending
# name_counts = unmatched.value_counts()

# # Convert to list (most frequent first)
# ordered_unmatched_list = name_counts.index.tolist()

# # # Print for copy-paste
# # for name in ordered_unmatched_list:
# #     print(name)

In [ ]:
# unmatched.describe()

count                   945
unique                  377
top       Meligeni Alves F.
freq                     10
dtype: object

In [22]:
import re
import time
import pandas as pd
import numpy as np
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException

# Sample setup to extract unique players
unique_players = pd.unique(df[['player_A_full', 'player_B_full']].values.ravel())
# unique_players = np.append(pd.unique(df[['player_A_full', 'player_B_full']].values.ravel())[:5], "Novak Djokovic")

# Chrome setup
options = Options()
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument("--disable-extensions")
options.add_argument("--disable-plugins")
options.add_argument("--disable-images")

driver = webdriver.Chrome(options=options)
driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")

# Handle cookie consent ONCE at the start
driver.get("https://www.ultimatetennisstatistics.com/")
time.sleep(3)

try:
    consent_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//button[contains(@class, 'consent')]"))
    )
    consent_button.click()
    time.sleep(2)
except Exception:
    pass

def normalize_name(name):
    import unicodedata
    normalized = unicodedata.normalize('NFD', name)
    normalized = ''.join(c for c in normalized if unicodedata.category(c) != 'Mn')
    return normalized.lower().strip()

def find_and_click_player(driver, search_box, target_player_name):
    try:
        WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, ".ui-menu-item"))
        )
        suggestions = driver.find_elements(By.CSS_SELECTOR, ".ui-menu-item")
        target_normalized = normalize_name(target_player_name)
        for suggestion in suggestions:
            suggestion_text = suggestion.text.strip()
            player_name_in_suggestion = suggestion_text.split('(')[0].strip()
            suggestion_normalized = normalize_name(player_name_in_suggestion)
            if suggestion_normalized == target_normalized:
                try:
                    driver.execute_script("arguments[0].click();", suggestion)
                    return True
                except:
                    try:
                        from selenium.webdriver.common.action_chains import ActionChains
                        actions = ActionChains(driver)
                        actions.move_to_element(suggestion).click().perform()
                        return True
                    except:
                        try:
                            suggestion.click()
                            return True
                        except:
                            continue
    except TimeoutException:
        pass

    try:
        search_box.send_keys(Keys.ARROW_DOWN)
        time.sleep(0.5)
        search_box.send_keys(Keys.ENTER)
        return True
    except:
        pass

    try:
        search_box.send_keys(Keys.ENTER)
        return True
    except:
        pass

    return False

# def wait_for_player_page_load(driver, player_name, max_attempts=5):
#     for _ in range(max_attempts):
#         try:
#             current_url = driver.current_url
#             if '#google_vignette' in current_url:
#                 return False
#             if "playerId=" in current_url or "/playerProfile" in current_url:
#                 time.sleep(2)
#                 return True
#             WebDriverWait(driver, 5).until(
#                 lambda d: d.execute_script("return document.readyState") == "complete"
#             )
#             time.sleep(2)
#         except:
#             time.sleep(2)
#     return False

def wait_for_player_page_load(driver, player_name, max_attempts=5):
    for _ in range(max_attempts):
        try:
            current_url = driver.current_url
            if '#google_vignette' in current_url:
                return False
            if "playerId=" in current_url or "/playerProfile" in current_url:
                return True
            WebDriverWait(driver, 5).until(
                lambda d: d.execute_script("return document.readyState") == "complete"
            )
            time.sleep(1)
        except:
            time.sleep(1)
    return False


def extract_player_data(driver, player_name):
    player_info = {'name': player_name, 'scraped_url': driver.current_url}
    try:
        WebDriverWait(driver, 5).until(
            lambda d: d.execute_script("return document.readyState") == "complete"
        )

        tables_found = False
        table_selectors = ["table", ".table", "#playerTable", ".player-table", ".stats-table", ".data-table"]

        for selector in table_selectors:
            try:
                tables = driver.find_elements(By.CSS_SELECTOR, selector)
                if tables:
                    for table in tables:
                        rows = table.find_elements(By.TAG_NAME, "tr")
                        for row in rows:
                            cells = row.find_elements(By.TAG_NAME, "td")
                            if len(cells) >= 2:
                                key = cells[0].text.strip()
                                value = cells[1].text.strip()
                                if key and value:
                                    clean_key = key.lower().replace(' ', '_').replace(':', '').replace('(', '').replace(')', '')
                                    player_info[clean_key] = value
                                    tables_found = True
                    if tables_found:
                        break
            except:
                continue

        if not tables_found:
            try:
                info_selectors = [".player-info", ".profile-info", ".stats-info", ".tab-content", ".content"]
                for selector in info_selectors:
                    try:
                        info_elements = driver.find_elements(By.CSS_SELECTOR, selector)
                        for element in info_elements:
                            text_content = element.text.strip()
                            # if 'H2H %' in text_content:
                            #     # Use regex to find everything up to and including "H2H %" line
                            #     match = re.search(r'(.*?H2H %\s*\d+(\.\d+)?%)', text_content, re.DOTALL)
                            #     if match:
                            #         player_info['content_summary'] = match.group(1).strip()
                            #     else:
                            #         player_info['content_summary'] = text_content  # fallback
                            # else:
                            #     player_info['content_summary'] = text_content
                            if text_content and len(text_content) > 50:
                                h2h_idx = text_content.find("H2H %")
                                if h2h_idx != -1:
                                    # Take everything up to and including the line containing H2H %
                                    lines = text_content[:h2h_idx].splitlines()
                                    final_line = text_content[h2h_idx:].splitlines()[0]
                                    summary = "\n".join(lines + [final_line])
                                    player_info['content_summary'] = summary.strip()
                                    player_info['has_h2h'] = True
                                else:
                                    # No H2H at all
                                    player_info['content_summary'] = text_content.strip()
                                    player_info['has_h2h'] = False
                                break
                    except:
                        continue
            except:
                pass

        try:
            page_title = driver.title
            if page_title and player_name.split()[0] in page_title:
                player_info['page_title'] = page_title
        except:
            pass

        return player_info

    except Exception as extraction_error:
        player_info['error'] = str(extraction_error)
        return player_info

players_data = []

for player in tqdm(unique_players, desc="Scraping Players", dynamic_ncols=True):
    try:
        try:
            search_box = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.ID, "player"))
            )
        except TimeoutException:
            driver.get("https://www.ultimatetennisstatistics.com/")
            time.sleep(2)
            search_box = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.ID, "player"))
            )

        search_box.clear()
        time.sleep(0.5)
        search_box.send_keys(player)
        time.sleep(1.5)

        click_success = find_and_click_player(driver, search_box, player)
        if not click_success:
            continue

        page_loaded = wait_for_player_page_load(driver, player)

        # 👇 NEW fix: retry if google ad hijacked the page
        if not page_loaded or '#google_vignette' in driver.current_url:
            driver.get("https://www.ultimatetennisstatistics.com/")
            time.sleep(3)

            try:
                search_box = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "player")))
                search_box.clear()
                time.sleep(0.5)
                search_box.send_keys(player)
                time.sleep(1.5)
                click_success = find_and_click_player(driver, search_box, player)
                if not click_success:
                    continue

                page_loaded = wait_for_player_page_load(driver, player)
                if not page_loaded or '#google_vignette' in driver.current_url:
                    continue  # Skip if second attempt also failed

            except:
                continue

        player_info = extract_player_data(driver, player)

        if len(player_info) > 2:
            players_data.append(player_info)
        else:
            player_info['status'] = 'incomplete_data'
            players_data.append(player_info)

    except Exception as e:
        error_info = {
            'name': player,
            'error': str(e),
            'scraped_url': driver.current_url,
            'status': 'error'
        }
        players_data.append(error_info)

driver.quit()

# Build DataFrame
players_df = pd.DataFrame(players_data)
print(players_df.head())

Scraping Players: 100%|██████████| 476/476 [45:29<00:00,  5.73s/it]  


                name  \
0      Florian Mayer   
1   Santiago Giraldo   
2  Nikolay Davydenko   
3     Victor Hănescu   
4      Fabio Fognini   

                                                                        scraped_url  \
0  https://www.ultimatetennisstatistics.com/playerProfile?playerId=4252&tab=profile   
1  https://www.ultimatetennisstatistics.com/playerProfile?playerId=5046&tab=profile   
2  https://www.ultimatetennisstatistics.com/playerProfile?playerId=3786&tab=profile   
3  https://www.ultimatetennisstatistics.com/playerProfile?playerId=3812&tab=profile   
4  https://www.ultimatetennisstatistics.com/playerProfile?playerId=4921&tab=profile   

                                                                                                                                                                                                                                                                                                                                            

In [21]:
players_df.head(6)

,name,scraped_url,content_summary,has_h2h,page_title
0,Florian Mayer,https://www.ultimatetennisstatistics.com/playerProfile?playerId=4252&tab=profile,"Age 41 (05-10-1983)\nCountry Germany\nHeight 190 cm\nPlays Right-handed\nBackhand Two-handed\nFavorite Surface Grass 28%\nTurned Pro 2001\nSeasons 15\nRetired 2018\nPrize Money US$7,278,992\nWikipedia Wikipedia\n Titles 2\nTeam Cups 2\n Best Rank 18 (06-06-2011)\nBest Elo Rank 22 (14-11-2011)\nPeak Elo Rating 2044 (14-11-2011)\nGOAT Rank 286 (7)\n Best Season 2011\nLast Appearance 27-08-2018\nUS Open Hard R128\n Overall\n48.2% (243-261)\nHard\n43.5% (101-131)\nClay\n50.0% (100-100)\n1\nGrass\n59.4% (38-26)\n1\nCarpet\n42.9% (3-4)\n\nH2H\n18\n6\n39\nH2H % 33.3%",True,Ultimate Tennis Statistics - Florian Mayer
1,Santiago Giraldo,https://www.ultimatetennisstatistics.com/playerProfile?playerId=5046&tab=profile,"Age 37 (27-11-1987)\nCountry Colombia\nHeight 188 cm\nPlays Right-handed\nBackhand Two-handed\nFavorite Surface Clay 22%\nTurned Pro 2006\nSeasons 15\nRetired 2019\nPrize Money $4,541,251\nWikipedia Wikipedia\n Best Rank 28 (29-09-2014)\nBest Elo Rank 32 (11-05-2014)\nPeak Elo Rating 1970 (10-06-2012)\nGOAT Rank 647 (1)\n Best Season 2014\nLast Appearance 26-08-2019\nUS Open Hard R128\n Overall\n44.7% (166-205)\nHard\n38.6% (61-97)\nClay\n50.8% (92-89)\nGrass\n43.3% (13-17)\nCarpet\n0.0% (0-2)\n\nH2H\n10\n6\n31\nH2H % 27.7%",True,Ultimate Tennis Statistics - Santiago Giraldo
2,Nikolay Davydenko,https://www.ultimatetennisstatistics.com/playerProfile?playerId=3786&tab=profile,"Age 44 (02-06-1981)\nCountry Russian Federation\nHeight 178 cm\nPlays Right-handed\nBackhand Two-handed\nFavorite Surface Carpet 52%\nTurned Pro 1999\nSeasons 15\nRetired 2014\nPrize Money US$16,186,480 35th all-time leader in earnings\nWikipedia Wikipedia\n Titles 21\nTour Finals 1\nMasters 3\nDavis Cups 1\n Best Rank 3 (06-11-2006)\nBest Elo Rank 5 (21-04-2008)\nPeak Elo Rating 2291 (01-02-2010)\nGOAT Rank 52 (76)\n Best Season 2006\nLast Appearance 26-05-2014\nRoland Garros Clay R128\n Overall\n59.4% (482-329)\nHard\n59.4% (253-173)\n8\nClay\n61.1% (185-118)\n10\nGrass\n33.3% (10-20)\nCarpet\n65.4% (34-18)\n3\n\nH2H\n68\n9\n33\nH2H % 65.9%",True,Ultimate Tennis Statistics - Nikolay Davydenko
3,Victor Hănescu,https://www.ultimatetennisstatistics.com/playerProfile?playerId=3812&tab=profile,"Age 43 (21-07-1981)\nCountry Romania\nHeight 198 cm\nPlays Right-handed\nBackhand One-handed\nFavorite Surface Clay 37%\nTurned Pro 1999\nSeasons 14\nRetired 2015\nPrize Money $4,297,051\nWikipedia Wikipedia\n Titles 1\n Best Rank 26 (06-07-2009)\nBest Elo Rank 41 (20-06-2010)\nPeak Elo Rating 1958 (07-06-2010)\nGOAT Rank 486 (2)\n Best Season 2008\nLast Appearance 12-03-2015\nIndian Wells Masters Hard R64\n Overall\n45.2% (201-244)\nHard\n34.2% (53-102)\nClay\n54.2% (130-110)\n1\nGrass\n40.6% (13-19)\nCarpet\n31.3% (5-11)\n\nH2H\n13\n6\n41\nH2H % 26.7%",True,Ultimate Tennis Statistics - Victor Hanescu
4,Fabio Fognini,https://www.ultimatetennisstatistics.com/playerProfile?playerId=4921&tab=profile,"Age 38 (24-05-1987)\nCountry Italy\nHeight 178 cm\nPlays Right-handed\nBackhand Two-handed\nFavorite Surface Clay 20%\nTurned Pro 2004\nSeasons 20\nActive Yes\nPrize Money US$16,673,573 34th all-time leader in earnings\nWikipedia Wikipedia\nWebsite FabioFognini.eu\n Titles 9\nMasters 1\n Current Rank 91 (637)\nBest Rank 9 (15-07-2019)\nCurrent Elo Rank 89 (1833)\nBest Elo Rank 14 (11-04-2014)\nPeak Elo Rating 2112 (20-04-2014)\nGOAT Rank 167 (18)\n Best Season 2019\nLast Appearance 04-11-2024\nBelgrade Hard R32\n Overall\n52.3% (425-388)\nHard\n46.2% (160-186)\n1\nClay\n57.2% (238-178)\n8\nGrass\n52.1% (25-23)\nCarpet\n0.0% (0-1)\n\nH2H\n41\n12\n48\nH2H % 46.5%",True,Ultimate Tennis Statistics - Fabio Fognini
5,Novak Djokovic,https://www.ultimatetennisstatistics.com/playerProfile?playerId=4920&tab=profile,"Age 38 (22-05-1987)\nCountry Serbia\nBirthplace Belgrade, Serbia\nResidence Monte Carlo, Monaco\nHeight 188 cm\nWeight 

In [ ]:
# players_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 475 entries, 0 to 474
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   name             474 non-null    object
 1   scraped_url      475 non-null    object
 2   content_summary  474 non-null    object
 3   page_title       453 non-null    object
 4   error            1 non-null      object
 5   status           1 non-null      object
dtypes: object(6)
memory usage: 22.4+ KB


In [27]:
print(players_df[players_df['name'] == 'Novak Djokovic']['content_summary'])

115    Age 38 (22-05-1987)\nCountry Serbia\nBirthplace Belgrade, Serbia\nResidence Monte Carlo, Monaco\nHeight 188 cm\nWeight 77 kg\nPlays Right-handed\nBackhand Two-handed\nFavorite Surface Hard 14%\nCoach Andy Murray\nTurned Pro 2003\nSeasons 21\nActive Yes\nPrize Money US$ 156,136,601 * All-time leader in earnings\nWikipedia Wikipedia\nWebsite novakdjokovic.com\nFacebook djokovic.official\nTwitter @DjokerNole\nNicknames Nole, Djoker\n  Titles 99\nGrand Slams 24\nTour Finals 7\nMasters 40\nOlympics 1\nDavis Cups 1\nTeam Cups 1\n  Current Rank 7 (3910)\nBest Rank 1 (04-07-2011)\nCurrent Elo Rank 2 (2347)\nBest Elo Rank 1 (21-03-2011)\nPeak Elo Rating 2629 (01-02-2016)\nGOAT Rank 1 (1120)\nWeeks at No. 1 428\n  Best Season 2015\nLast Appearance 02-10-2024\nShanghai Masters Hard F\n  Overall\n83.6% (1123-220)\nHard\n84.8% (708-127)\n71\nClay\n80.6% (286-69)\n20\nGrass\n85.7% (120-20)\n8\nCarpet\n69.2% (9-4)\n\nH2H\n128\n4\n4\nH2H % 95.6%
Name: content_summary, dtype: object


In [ ]:
# players_df[players_df['name'] == 'Santiago Giraldo']['content_summary']

1    Age 37 (27-11-1987)\nCountry Colombia\nHeight 188 cm\nPlays Right-handed\nBackhand Two-handed\nFavorite Surface Clay 22%\nTurned Pro 2006\nSeasons 15\nRetired 2019\nPrize Money $4,541,251\nWikipedia Wikipedia\n  Best Rank 28 (29-09-2014)\nBest Elo Rank 32 (11-05-2014)\nPeak Elo Rating 1970 (10-06-2012)\nGOAT Rank 647 (1)\n  Best Season 2014\nLast Appearance 26-08-2019\nUS Open Hard R128\n  Overall\n44.7% (166-205)\nHard\n38.6% (61-97)\nClay\n50.8% (92-89)\nGrass\n43.3% (13-17)\nCarpet\n0.0% (0-2)\n\nH2H\n10\n6\n31\nH2H % 27.7%\n
Name: content_summary, dtype: object

### Saving the created players_df

In [ ]:
# import os
# print(os.getcwd())

c:\Users\user\Desktop\Thesis\Thesis_code\DSB_Thesis


In [23]:
# Saving the scraped file as csv 
players_df.to_csv('data/players_df_final.csv', index=False)

## Getting the player dataset

In [24]:
# final_players_df = pd.read_csv('data/players_df.csv')
final_players_df = pd.read_csv('data/players_df_final.csv')

In [25]:
final_players_df.head()

,name,scraped_url,content_summary,has_h2h,page_title,error,status
0,Florian Mayer,https://www.ultimatetennisstatistics.com/playerProfile?playerId=4252&tab=profile,"Age 41 (05-10-1983)\nCountry Germany\nHeight 190 cm\nPlays Right-handed\nBackhand Two-handed\nFavorite Surface Grass 28%\nTurned Pro 2001\nSeasons 15\nRetired 2018\nPrize Money US$7,278,992\nWikipedia Wikipedia\n Titles 2\nTeam Cups 2\n Best Rank 18 (06-06-2011)\nBest Elo Rank 22 (14-11-2011)\nPeak Elo Rating 2044 (14-11-2011)\nGOAT Rank 286 (7)\n Best Season 2011\nLast Appearance 27-08-2018\nUS Open Hard R128\n Overall\n48.2% (243-261)\nHard\n43.5% (101-131)\nClay\n50.0% (100-100)\n1\nGrass\n59.4% (38-26)\n1\nCarpet\n42.9% (3-4)\n\nH2H\n18\n6\n39\nH2H % 33.3%",True,Ultimate Tennis Statistics - Florian Mayer,NaN,NaN
1,Santiago Giraldo,https://www.ultimatetennisstatistics.com/playerProfile?playerId=5046&tab=profile,"Age 37 (27-11-1987)\nCountry Colombia\nHeight 188 cm\nPlays Right-handed\nBackhand Two-handed\nFavorite Surface Clay 22%\nTurned Pro 2006\nSeasons 15\nRetired 2019\nPrize Money $4,541,251\nWikipedia Wikipedia\n Best Rank 28 (29-09-2014)\nBest Elo Rank 32 (11-05-2014)\nPeak Elo Rating 1970 (10-06-2012)\nGOAT Rank 647 (1)\n Best Season 2014\nLast Appearance 26-08-2019\nUS Open Hard R128\n Overall\n44.7% (166-205)\nHard\n38.6% (61-97)\nClay\n50.8% (92-89)\nGrass\n43.3% (13-17)\nCarpet\n0.0% (0-2)\n\nH2H\n10\n6\n31\nH2H % 27.7%",True,Ultimate Tennis Statistics - Santiago Giraldo,NaN,NaN
2,Nikolay Davydenko,https://www.ultimatetennisstatistics.com/playerProfile?playerId=3786&tab=profile,"Age 44 (02-06-1981)\nCountry Russian Federation\nHeight 178 cm\nPlays Right-handed\nBackhand Two-handed\nFavorite Surface Carpet 52%\nTurned Pro 1999\nSeasons 15\nRetired 2014\nPrize Money US$16,186,480 35th all-time leader in earnings\nWikipedia Wikipedia\n Titles 21\nTour Finals 1\nMasters 3\nDavis Cups 1\n Best Rank 3 (06-11-2006)\nBest Elo Rank 5 (21-04-2008)\nPeak Elo Rating 2291 (01-02-2010)\nGOAT Rank 52 (76)\n Best Season 2006\nLast Appearance 26-05-2014\nRoland Garros Clay R128\n Overall\n59.4% (482-329)\nHard\n59.4% (253-173)\n8\nClay\n61.1% (185-118)\n10\nGrass\n33.3% (10-20)\nCarpet\n65.4% (34-18)\n3\n\nH2H\n68\n9\n33\nH2H % 65.9%",True,Ultimate Tennis Statistics - Nikolay Davydenko,NaN,NaN
3,Victor Hănescu,https://www.ultimatetennisstatistics.com/playerProfile?playerId=3812&tab=profile,"Age 43 (21-07-1981)\nCountry Romania\nHeight 198 cm\nPlays Right-handed\nBackhand One-handed\nFavorite Surface Clay 37%\nTurned Pro 1999\nSeasons 14\nRetired 2015\nPrize Money $4,297,051\nWikipedia Wikipedia\n Titles 1\n Best Rank 26 (06-07-2009)\nBest Elo Rank 41 (20-06-2010)\nPeak Elo Rating 1958 (07-06-2010)\nGOAT Rank 486 (2)\n Best Season 2008\nLast Appearance 12-03-2015\nIndian Wells Masters Hard R64\n Overall\n45.2% (201-244)\nHard\n34.2% (53-102)\nClay\n54.2% (130-110)\n1\nGrass\n40.6% (13-19)\nCarpet\n31.3% (5-11)\n\nH2H\n13\n6\n41\nH2H % 26.7%",True,Ultimate Tennis Statistics - Victor Hanescu,NaN,NaN
4,Fabio Fognini,https://www.ultimatetennisstatistics.com/playerProfile?playerId=4921&tab=profile,"Age 38 (24-05-1987)\nCountry Italy\nHeight 178 cm\nPlays Right-handed\nBackhand Two-handed\nFavorite Surface Clay 20%\nTurned Pro 2004\nSeasons 20\nActive Yes\nPrize Money US$16,673,573 34th all-time leader in earnings\nWikipedia Wikipedia\nWebsite FabioFognini.eu\n Titles 9\nMasters 1\n Current Rank 91 (637)\nBest Rank 9 (15-07-2019)\nCurrent Elo Rank 89 (1833)\nBest Elo Rank 14 (11-04-2014)\nPeak Elo Rating 2112 (20-04-2014)\nGOAT Rank 167 (18)\n Best Season 2019\nLast Appearance 04-11-2024\nBelgrade Hard R32\n Overall\n52.3% (425-388)\nHard\n46.2% (160-186)\n1\nClay\n57.2% (238-178)\n8\nGrass\n52.1% (25-23)\nCarpet\n0.0% (0-1)\n\nH2H\n41\n12\n48\nH2H % 46.5%",True,Ultimate Tennis Statistics - Fabio Fognini,NaN,NaN


In [ ]:
import pandas as pd
import re


# Function to extract fields using regex
def extract_fields(summary):
    fields = {}

    if not isinstance(summary, str):
        return fields  # Skip non-string inputs like NaN
    
    patterns = {
        "Age": r"Age (\d+) \(([^\)]+)\)",
        "Country": r"Country ([^\n]+)",
        "Birthplace": r"Birthplace ([^\n]+)",
        "Residence": r"Residence ([^\n]+)",
        "Height": r"Height ([^\n]+)",
        "Weight": r"Weight ([^\n]+)",
        "Plays": r"Plays ([^\n]+)",
        "Backhand": r"Backhand ([^\n]+)",
        "Favorite Surface": r"Favorite Surface ([^\n]+)",
        "Coach": r"Coach ([^\n]+)",
        "Turned Pro": r"Turned Pro (\d+)",
        "Seasons": r"Seasons (\d+)",
        "Active": r"Active ([^\n]+)",
        "Retired": r"Retired (\d+)",
        "Prize Money": r"Prize Money ([^\n]+)",
        "Titles": r"Titles (\d+)",
        "Grand Slams": r"Grand Slams (\d+)",
        "Tour Finals": r"Tour Finals (\d+)",
        "Masters": r"Masters (\d+)",
        "Olympics": r"Olympics (\d+)",
        "Davis Cups": r"Davis Cups (\d+)",
        "Team Cups": r"Team Cups (\d+)",
        "Current Rank": r"Current Rank ([^\n]+)",
        "Best Rank": r"Best Rank ([^\n]+)",
        "Current Elo Rank": r"Current Elo Rank ([^\n]+)",
        "Best Elo Rank": r"Best Elo Rank ([^\n]+)",
        "Peak Elo Rating": r"Peak Elo Rating ([^\n]+)",
        "GOAT Rank": r"GOAT Rank ([^\n]+)",
        "Weeks at No. 1": r"Weeks at No. 1 ([^\n]+)",
        "Best Season": r"Best Season ([^\n]+)",
        
        # Overall and surfaces
        "Overall Win %": r"Overall\n([\d\.]+)% \(([0-9\-]+)\)",
        "Hard Win %": r"Hard\n([\d\.]+)% \(([0-9\-]+)\)\n(\d+)",
        "Clay Win %": r"Clay\n([\d\.]+)% \(([0-9\-]+)\)\n(\d+)",
        "Grass Win %": r"Grass\n([\d\.]+)% \(([0-9\-]+)\)\n(\d+)",
        "Carpet Win %": r"Carpet\n([\d\.]+)% \(([0-9\-]+)\)\n(\d+)",

        # H2H stats
        "H2H Wins": r"H2H\n(\d+)",
        "H2H Draws": r"H2H\n\d+\n(\d+)",
        "H2H Losses": r"H2H\n\d+\n\d+\n(\d+)",
        "H2H %": r"H2H % ([\d\.]+)%"
    }
    
    for key, pattern in patterns.items():
        match = re.search(pattern, summary)
        fields[key] = match.group(1) if match else None
        
    return fields

# Apply extraction to each player
extracted = final_players_df['content_summary'].apply(extract_fields)
extracted_df = pd.json_normalize(extracted)

# Combine with original dataset
df_final = pd.concat([final_players_df, extracted_df], axis=1)

print("Extraction complete. Cleaned dataset saved as 'cleaned_players_dataset.csv'")

Extraction complete. Cleaned dataset saved as 'cleaned_players_dataset.csv'


In [45]:
df_final[df_final['name'] == 'Carlos Alcaraz']

,name,scraped_url,content_summary,has_h2h,page_title,error,status,Age,Country,Birthplace,Residence,Height,Weight,Plays,Backhand,Favorite Surface,Coach,Turned Pro,Seasons,Active,Retired,Prize Money,Titles,Grand Slams,Tour Finals,Masters,Olympics,Davis Cups,Team Cups,Current Rank,Best Rank,Current Elo Rank,Best Elo Rank,Peak Elo Rating,GOAT Rank,Weeks at No. 1,Best Season,Overall Win %,Hard Win %,Clay Win %,Grass Win %,Carpet Win %,H2H Wins,H2H Draws,H2H Losses,H2H %
395,Carlos Alcaraz,https://www.ultimatetennisstatistics.com/playerProfile?playerId=52602&tab=profile,"Age 22 (05-05-2003)\nCountry Spain\nHeight 183 cm\nPlays Right-handed\nBackhand Two-handed\nFavorite Surface Grass 35%\nTurned Pro 2018\nSeasons 5\nActive Yes\nPrize Money US$5,452,072\nWikipedia Wikipedia\n Titles 16\nGrand Slams 4\nMasters 5\n Current Rank 3 (7010)\nBest Rank 1 (12-09-2022)\nCurrent Elo Rank 3 (2280)\nBest Elo Rank 1 (15-07-2024)\nPeak Elo Rating 2348 (15-07-2024)\nGOAT Rank 27 (137)\nWeeks at No. 1 36\n Best Season 2023\nLast Appearance 11-11-2024\nTour Finals Hard (i) RR\n Overall\n78.2% (201-56)\nHard\n73.5% (97-35)\n5\nClay\n81.6% (80-18)\n8\nGrass\n88.9% (24-3)\n3\n\nH2H\n27\n0\n3\nH2H % 90.0%",True,Ultimate Tennis Statistics - Carlos Alcaraz Garfia,NaN,NaN,22,Spain,None,None,183 cm,None,Right-handed,Two-handed,Grass 35%,None,2018,5,Yes,None,"US$5,452,072",16,4,None,5,None,None,None,3 (7010),1 (12-09-2022),3 (2280),1 (15-07-2024),2348 (15-07-2024),27 (137),36,2023,78.2,73.5,81.6,88.9,None,27,0,3,90.0


In [46]:
df_final.drop(columns= ['scraped_url', 'content_summary', 'has_h2h', 'page_title', 'error', 'status'], axis=1, inplace=True)

In [50]:
df_final.head()

,name,Age,Country,Birthplace,Residence,Height,Weight,Plays,Backhand,Favorite Surface,Coach,Turned Pro,Seasons,Active,Retired,Prize Money,Titles,Grand Slams,Tour Finals,Masters,Olympics,Davis Cups,Team Cups,Current Rank,Best Rank,Current Elo Rank,Best Elo Rank,Peak Elo Rating,GOAT Rank,Weeks at No. 1,Best Season,Overall Win %,Hard Win %,Clay Win %,Grass Win %,Carpet Win %,H2H Wins,H2H Draws,H2H Losses,H2H %
0,Florian Mayer,41,Germany,None,None,190 cm,None,Right-handed,Two-handed,Grass 28%,None,2001,15,None,2018,"US$7,278,992",2,None,None,None,None,None,2,None,18 (06-06-2011),None,22 (14-11-2011),2044 (14-11-2011),286 (7),None,2011,48.2,None,50.0,59.4,None,18,6,39,33.3
1,Santiago Giraldo,37,Colombia,None,None,188 cm,None,Right-handed,Two-handed,Clay 22%,None,2006,15,None,2019,"$4,541,251",None,None,None,None,None,None,None,None,28 (29-09-2014),None,32 (11-05-2014),1970 (10-06-2012),647 (1),None,2014,44.7,None,None,None,None,10,6,31,27.7
2,Nikolay Davydenko,44,Russian Federation,None,None,178 cm,None,Right-handed,Two-handed,Carpet 52%,None,1999,15,None,2014,"US$16,186,480 35th all-time leader in earnings",21,None,1,3,None,1,None,None,3 (06-11-2006),None,5 (21-04-2008),2291 (01-02-2010),52 (76),None,2006,59.4,59.4,61.1,None,65.4,68,9,33,65.9
3,Victor Hănescu,43,Romania,None,None,198 cm,None,Right-handed,One-handed,Clay 37%,None,1999,14,None,2015,"$4,297,051",1,None,None,None,None,None,None,None,26 (06-07-2009),None,41 (20-06-2010),1958 (07-06-2010),486 (2),None,2008,45.2,None,54.2,None,None,13,6,41,26.7
4,Fabio Fognini,38,Italy,None,None,178 cm,None,Right-handed,Two-handed,Clay 20%,None,2004,20,Yes,None,"US$16,673,573 34th all-time leader in earnings",9,None,None,1,None,None,None,91 (637),9 (15-07-2019),89 (1833),14 (11-04-2014),2112 (20-04-2014),167 (18),None,2019,52.3,46.2,57.2,None,None,41,12,48,46.5


In [47]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 476 entries, 0 to 475
Data columns (total 40 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   name              475 non-null    object
 1   Age               465 non-null    object
 2   Country           475 non-null    object
 3   Birthplace        30 non-null     object
 4   Residence         30 non-null     object
 5   Height            449 non-null    object
 6   Weight            29 non-null     object
 7   Plays             465 non-null    object
 8   Backhand          423 non-null    object
 9   Favorite Surface  433 non-null    object
 10  Coach             30 non-null     object
 11  Turned Pro        372 non-null    object
 12  Seasons           466 non-null    object
 13  Active            196 non-null    object
 14  Retired           269 non-null    object
 15  Prize Money       430 non-null    object
 16  Titles            180 non-null    object
 17  Grand Slams     

In [ ]:
# # Save the cleaned dataset
# df_final.to_csv("data/cleaned_players_dataset_final.csv", index=False)